The goal of this assignment is to implement your own logistic regression classifier. You will:

Extract features from Amazon product reviews.
Convert an SFrame into a NumPy array.
Implement the link function for logistic regression.
Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
Implement gradient ascent.
Given a set of coefficients, predict sentiments.
Compute classification accuracy for the logistic regression model.

In [3]:
import numpy as np
import pandas as pd
from sklearn import linear_model 

products = pd.read_csv('/Users/kevinwang/Documents/Coursera/Machine Learning - Classification/Week2/amazon_baby_subset.csv')  

## Load review dataset

For this assignment, we will use a subset of the Amazon product review dataset. The subset was chosen to contain similar numbers of positive and negative reviews, as the original dataset consisted primarily of positive reviews.

Load the dataset into a data frame named products. One column of this dataset is sentiment, corresponding to the class label with +1 indicating a review with positive sentiment and -1 for negative sentiment.

Let us quickly explore more of this dataset. The name column indicates the name of the product. Try listing the name of the first 10 products in the dataset.

After that, try counting the number of positive and negative reviews.

Note: For this assignment, we eliminated class imbalance by choosing a subset of the data with a similar number of positive and negative reviews.

In [12]:
print(products[0:10])
num_pos = sum(products['sentiment']==1)
num_neg = sum(products['sentiment']==-1)
print(num_pos)
print(num_neg)

                                                name  \
0  Stop Pacifier Sucking without tears with Thumb...   
1    Nature's Lullabies Second Year Sticker Calendar   
2    Nature's Lullabies Second Year Sticker Calendar   
3                        Lamaze Peekaboo, I Love You   
4  SoftPlay Peek-A-Boo Where's Elmo A Children's ...   
5                          Our Baby Girl Memory Book   
6  Hunnt&reg; Falling Flowers and Birds Kids Nurs...   
7  Blessed By Pope Benedict XVI Divine Mercy Full...   
8  Cloth Diaper Pins Stainless Steel Traditional ...   
9  Cloth Diaper Pins Stainless Steel Traditional ...   

                                              review  rating  sentiment  
0  All of my kids have cried non-stop when I trie...       5          1  
1  We wanted to get something to keep track of ou...       5          1  
2  My daughter had her 1st baby over a year ago. ...       5          1  
3  One of baby's first and favorite books, and it...       4          1  
4  Very cute 

## Apply text cleaning on the review data

In this section, we will perform some simple feature cleaning using data frames. The last assignment used all words in building bag-of-words features, but here we limit ourselves to 193 words (for simplicity). We compiled a list of 193 most frequent words into the JSON file named important_words.json. Load the words into a list important_words.

Let us perform 2 simple data transformations:

Remove punctuation
Compute word counts (only for important_words)
We start with the first item as follows:

If your tool supports it, fill n/a values in the review column with empty strings. The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the review columns as follows:

In [22]:
important_words = pd.read_json('/Users/kevinwang/Documents/Coursera/Machine Learning - Classification/Week2/important_words.json')
#print(important_words)
#type(important_words)   #df
#print(important_words[0])
#type(important_words[0])   #series

In [29]:
products = products.fillna({'review':''})  # fill empty string to replace N/A's in 'review' column

Write a function remove_punctuation that takes a line of text and removes all punctuation from that text. The function should be analogous to the following Python code:

In [30]:
def remove_punctuation(text):
    import string
    return text.translate(str.maketrans('','',string.punctuation))    # nature's will change to natures

Apply the remove_punctuation function on every element of the review column and assign the result to the new column review_clean. Note. Many data frame packages support apply operation for this type of task. Consult appropriate manuals.

In [31]:
products['review_clean'] = products['review'].apply(remove_punctuation)

Now we proceed with the second item. For each word in important_words, we compute a count for the number of times the word occurs in the review. We will store this count in a separate column (one for each word). The result of this feature processing is a single column for each word in important_words which keeps a count of the number of times the respective word occurs in the review text.

Note: There are several ways of doing this. One way is to create an anonymous function that counts the occurrence of a particular word and apply it to every element in the review_clean column. Repeat this step for every word in important_words. Your code should be analogous to the following:

In [32]:
for word in important_words[0]:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

The data frame products should contain one column for each of the 193 important_words. As an example, the column perfect contains a count of the number of times the word perfect occurs in each of the reviews.

Now, write some code to compute the number of product reviews that contain the word 'perfect'.

Hint:

First create a column called 'contains_perfect' which is set to 1 if the count of the word perfect (stored in column perfect is >= 1.
Sum the number of 1s in the column contains_perfect.

In [38]:
products['countains_perfect'] = products['perfect'].apply(lambda x : 1 if x >= 1 else 0)
#print(products['countains_perfect'])
sum(products['countains_perfect'])

2955

Quiz Question. How many reviews contain the word perfect?

Answer: 2955

## Convert data frame to multi-dimensional array

It is now time to convert our data frame to a multi-dimensional array. Look for a package that provides a highly optimized matrix operations. In the case of Python, NumPy is a good choice.

Write a function that extracts columns from a data frame and converts them into a multi-dimensional array. We plan to use them throughout the course, so make sure to get this function right.

The function should accept three parameters:

dataframe: a data frame to be converted
features: a list of string, containing the names of the columns that are used as features.
label: a string, containing the name of the single column that is used as class labels.
The function should return two values:

one 2D array for features
one 1D array for class labels
The function should do the following:

Prepend a new column constant to dataframe and fill it with 1's. This column takes account of the intercept term. Make sure that the constant column appears first in the data frame.
Prepend a string 'constant' to the list features. Make sure the string 'constant' appears first in the list.
Extract columns in dataframe whose names appear in the list features.
Convert the extracted columns into a 2D array using a function in the data frame library. If you are using Pandas, you would use as_matrix() function.
Extract the single column in dataframe whose name corresponds to the string label.
Convert the column into a 1D array.
Return the 2D array and the 1D array.

In [57]:
def get_numpy_data(dataframe, features, label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    features_frame = dataframe[features]
    feature_matrix = features_frame.as_matrix()
    label_sarray = dataframe[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

Using the function written, extract two arrays feature_matrix and sentiment. The 2D array feature_matrix would contain the content of the columns given by the list important_words. The 1D array sentiment would contain the content of the column sentiment.

Quiz Question: How many features are there in the feature_matrix?

Answer: 194

Quiz Question: Assuming that the intercept is present, how does the number of features in feature_matrix relate to the number of features in the logistic regression model?

In [58]:
feature_matrix, sentiment = get_numpy_data(products, list(important_words[0]), 'sentiment')
len(get_numpy_data(products, list(important_words[0]), 'sentiment')[0][0])


/Users/kevinwang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
/Users/kevinwang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


194

## Estimating conditional probability with link function

Recall from lecture that the link function is given by

P(y=+1|xi,w)=1/(1+exp(−w*h(xi)))
where the feature vector h(xi) represents the word counts of important_words in the review xi. Write a function named predict_probability that implements the link function.

Take two parameters: feature_matrix and coefficients.
First compute the dot product of feature_matrix and coefficients.
Then compute the link function P(y=+1|x,w).
Return the predictions given by the link function.
Your code should be analogous to the following Python function:

In [74]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score = np.dot(feature_matrix, coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1/(1+np.exp(-score))
    
    # return predictions
    return predictions

## Compute derivative of log likelihood with respect to a single coefficient

Recall from lecture:  

∂L / ∂wj =∑ hj(xi)* (1[yi=+1]−P(yi=+1|xi,w)) 

This is derivative of log likelihood, not derivative of loss, so the larger the better, that's why we do gradient ascent later.

i is i-th data, j is j-th feature

We will now write a function feature_derivative that computes the derivative of log likelihood with respect to a single coefficient w_j. The function accepts two arguments:

errors: vector whose i-th value contains
1[yi=+1]−P(yi=+1|xi,w)

feature: vector whose i-th value contains
hj(xi)

This corresponds to the j-th column of feature_matrix.

The function should do the following:

Take two parameters errors and feature.
Compute the dot product of errors and feature.
Return the dot product. This is the derivative with respect to a single coefficient w_j.
Your code should be analogous to the following Python function:

In [68]:
def feature_derivative(errors, feature):       #derivative of log likelihood  
    # Compute the dot product of errors and feature
    derivative = np.dot(errors, feature)    #errors => (1,n)  features => (n, m)    derivative => (1,m)
        # Return the derivative
    return derivative

In the main lecture, our focus was on the likelihood. In the advanced optional video, however, we introduced a transformation of this likelihood---called the log-likelihood---that simplifies the derivation of the gradient and is more numerically stable. Due to its numerical stability, we will use the log-likelihood instead of the likelihood to assess the algorithm.

The log-likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):

ll(w)=∑((1[yi=+1]−1)w*h(wi)−ln(1+exp(−w*h(xi))))          

w => (m,1)  features => (n, m)

Write a function compute_log_likelihood that implements the equation. The function would be analogous to the following Python function:

In [69]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)    #(n,1)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))    #(1,1)
    return lp           

## Taking gradient steps

Now we are ready to implement our own logistic regression. All we have to do is to write a gradient ascent function that takes gradient steps towards the optimum.

Write a function logistic_regression to fit a logistic regression model using gradient ascent.

The function accepts the following parameters:

feature_matrix: 2D array of features
sentiment: 1D array of class labels
initial_coefficients: 1D array containing initial values of coefficients
step_size: a parameter controlling the size of the gradient steps
max_iter: number of iterations to run gradient ascent

The function returns the last set of coefficients after performing gradient ascent.

The function carries out the following steps:

Initialize vector coefficients to initial_coefficients.
Predict the class probability P(yi=+1|xi,w) using your predict_probability function and save it to variable predictions.
Compute indicator value for (yi=+1) by comparing sentiment against +1. Save it to variable indicator.
Compute the errors as difference between indicator and predictions. Save the errors to variable errors.
For each j-th coefficient, compute the per-coefficient derivative by calling feature_derivative with the j-th column of feature_matrix. Then increment the j-th coefficient by (step_size*derivative).
Once in a while, insert code to print out the log likelihood.
Repeat steps 2-6 for max_iter times.
At the end of day, your code should be analogous to the following Python function (with blanks filled in):

In [114]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions

        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:,j])    #d(Log_likelihood)/dwi      (1,1)

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j] = coefficients[j] + step_size * derivative

        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

Now, let us run the logistic regression solver with the parameters below:

feature_matrix = feature_matrix extracted 
sentiment = sentiment extracted
initial_coefficients = a 194-dimensional vector filled with zeros
step_size = 1e-7
max_iter = 301
Save the returned coefficients to variable coefficients.

Quiz question: As each iteration of gradient ascent passes, does the log likelihood increase or decrease?

Answer: Increase

In [115]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros((194,)), step_size=1e-7, max_iter=301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

## Predicting sentiments

Recall from lecture that class predictions for a data point x can be computed from the coefficients w using the following formula:

y = +1  if x*w>0

y = =1  if x*w≤0

Now, we write some code to compute class predictions. We do this in two steps:

First compute the scores using feature_matrix and coefficients using a dot product.
Then apply threshold 0 on the scores to compute the class predictions. Refer to the formula above.


In [84]:
scores = np.dot(feature_matrix, coefficients)   # feature_matrix => (n, m) coefficients => (m, 1)  scores => (n,1)

In [93]:
class_predict = np.array([1 if u > 0 else -1 for u in scores])    #array
sum(class_predict == 1)

25126

In [94]:
#class_predict = [1 if u > 0 else -1 for u in scores]     #list
#type(class_predict)
#sum(class_predict == 1)   #error

Quiz question: How many reviews were predicted to have positive sentiment?

Answer: 25126

## Measuring accuracy

We will now measure the classification accuracy of the model. Recall from the lecture that the classification accuracy can be computed as follows:

accuracy=# correctly classified data points / # total data points

In [119]:
sum(class_predict == sentiment)/len(class_predict)

0.7518653904130238

Quiz question: What is the accuracy of the model on predictions made above? (round to 2 digits of accuracy)

Answer:  0.7518653904130238

## Which words contribute most to positive & negative sentiments

Recall that in the earlier assignment, we were able to compute the "most positive words". These are words that correspond most strongly with positive reviews. In order to do this, we will first do the following:

Treat each coefficient as a tuple, i.e. (word, coefficient_value). The intercept has no corresponding word, so throw it out.
Sort all the (word, coefficient_value) tuples by coefficient_value in descending order. Save the sorted list of tuples to word_coefficient_tuples.
Your code should be analogous to the following:

Now, word_coefficient_tuples contains a sorted list of (word, coefficient_value) tuples. The first 10 elements in this list correspond to the words that are most positive.


In [120]:
#coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(list(important_words[0]), list(coefficients[1:]))]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)
print(word_coefficient_tuples[:10])
print()
print(word_coefficient_tuples[-10:])

[('great', 0.0665460841704577), ('love', 0.06589076292212327), ('easy', 0.0647945868025784), ('little', 0.04543562630842137), ('loves', 0.04497640139490604), ('well', 0.030135001092107084), ('perfect', 0.029739937104968462), ('old', 0.020077541034775385), ('nice', 0.018408707995268992), ('daughter', 0.017703199905701694)]

[('monitor', -0.024482100545891717), ('return', -0.026592778462247283), ('back', -0.027742697230661334), ('get', -0.02871155298019258), ('disappointed', -0.028978976142317068), ('even', -0.030051249236035804), ('work', -0.03306951529475273), ('money', -0.038982037286487116), ('product', -0.041511033392108904), ('would', -0.05386014844520314)]


## Ten "most positive" words

Compute the 10 words that have the most positive coefficient values. These words are associated with positive sentiment.

Quiz question: Which word is not present in the top 10 "most positive" words?

Answer:  [('great', 0.0665460841704577), ('love', 0.06589076292212327), ('easy', 0.0647945868025784), ('little', 0.04543562630842137), ('loves', 0.04497640139490604), ('well', 0.030135001092107084), ('perfect', 0.029739937104968462), ('old', 0.020077541034775385), ('nice', 0.018408707995268992), ('daughter', 0.017703199905701694)]


## Ten "most negative" words

Next, we repeat this exerciese on the 10 most negative words. That is, we compute the 10 words that have the most negative coefficient values. These words are associated with negative sentiment.

Quiz question: Which word is not present in the top 10 "most negative" words?

Answer: [('monitor', -0.024482100545891717), ('return', -0.026592778462247283), ('back', -0.027742697230661334), ('get', -0.02871155298019258), ('disappointed', -0.028978976142317068), ('even', -0.030051249236035804), ('work', -0.03306951529475273), ('money', -0.038982037286487116), ('product', -0.041511033392108904), ('would', -0.05386014844520314)]

